## Preprocessing

### Imports libraries

In [4]:
import os
import sys
sys.path.append(r"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification")

from tqdm import tqdm
from datetime import datetime
import shutil
import dicom2nifti
import pandas as pd

import brain_mets_classification.config as config
import brain_mets_classification.custom_funcs as funcs

In [5]:
# path to the sample n=30 directory
#sampleDirectoryPath = f"{config.path}/RgbBrainMetsSampleN30"

# path to all patients
sampleDirectoryPath = f"{config.path}/originalPatientFiles"

### Step 0: Copies files and creates new folders with the desired MRI sequences already renamed and converted into nifti files
Sequences that will be used for this project:
- axiale T2
- axiale Flair
- axiale T1 vor + nach KM
- sagittale T1 MPRage nach KM
- evtl. DWI (Diffsionswichtung) + ADC

In [6]:
# Creates a new folder within the directory for all the files of the desired MRI sequences
# now = datetime.now()
# timeFormatted = now.strftime("%Y-%m-%d %H:%M:%S")
# pathToPreprocessing0 = f"{config.path}/Rgb_Brain_Mets_Preprocessing#0_{timeFormatted}"
# os.mkdir(pathToPreprocessing0)
pathToPreprocessing0 = funcs.createNewPreprocessingStepFolder(step=0)

# list all the names of the folders within the directory
patientFolders = os.listdir(sampleDirectoryPath)

 # removes the .DS_Store file if it can be found within the directory
dsStore = ".DS_Store"
if dsStore in patientFolders:
    patientFolders.remove(dsStore)

patientsIDs = []
#print(patientFolders)


for folder in tqdm(patientFolders): # loops through the all folders for each patient

    # trying to catch all the ds_folders
    if dsStore in folder:
        continue
    
    mriSequences = []

    print(folder)

    patientID = folder.split(" - ")[1]

    # adds the patientID to the list of patientIDs if is hasn't been added before
    if patientID not in patientsIDs:
        patientsIDs.append(patientID)
    
    folderPath = os.path.join(sampleDirectoryPath, folder) #path to the content of each patient's folder
    os.chdir(folderPath)
    mriImagesFolders = os.listdir(os.getcwd())

    for count in range(0, len(mriImagesFolders)):
        mriImagesFolder = mriImagesFolders[count]
        if mriImagesFolder != config.dsStore:
            os.chdir(mriImagesFolder)
        else:
            continue   
        

        mriSequences = os.listdir(os.getcwd())
        # print(mriSequences)

        for sequence in mriSequences:

            pathToSequence = os.path.join(folderPath, mriImagesFolder, sequence)

            sequenceFiles = os.listdir(pathToSequence)
            if len(sequenceFiles) <= 3:
                continue

            #print(sequence)

            sequenceLC = sequence.lower()
            if any(whiteListSeq in sequenceLC for whiteListSeq in config.seq_whitelist) and not any(blackListSeq in sequenceLC for blackListSeq in config.seq_blacklist):
                # sequence fits both the whiteList as well as the blackList

                # create folder for patient
                funcs.createFolderForPatient(path=pathToPreprocessing0, patientID=patientID)
                
                

                #T1/T1CE
                if any(sequence in sequenceLC for sequence in config.T1list) and not any(subSequence in sequenceLC for subSequence in config.sub):
                    
                    # check if T1 is contrast enhanced (CE)
                    if any(ceSequence in sequenceLC for ceSequence in config.KMlist):
                        dicom2nifti.convert_directory(pathToSequence, f"{pathToPreprocessing0}/{patientID}")

                        try:
                            unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToPreprocessing0}/{patientID}")
                        except RuntimeError:
                            print(RuntimeError)
                        os.rename(unRenamedFile, f"{pathToPreprocessing0}/{patientID}/{patientID}_T1CE.nii.gz")
                        
                        # convert to nift, call the file [PATIENTID]_T1CE and move to Patient Folder

                    else:
                        dicom2nifti.convert_directory(pathToSequence, f"{pathToPreprocessing0}/{patientID}")
                        unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToPreprocessing0}/{patientID}")
                        os.rename(unRenamedFile, f"{pathToPreprocessing0}/{patientID}/{patientID}_T1.nii.gz")
                        # convert to nift and call the file [PATIENTID]_T1

                #T2
                if any(T2Sequence in sequenceLC for T2Sequence in config.T2list) and not any(T2SternSequence in sequenceLC for T2SternSequence in config.T2STERNlist) and not any(FlairSequence in sequenceLC for FlairSequence in config.FLAIRlist):
                    dicom2nifti.convert_directory(pathToSequence, f"{pathToPreprocessing0}/{patientID}")
                    unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToPreprocessing0}/{patientID}")
                    os.rename(unRenamedFile, f"{pathToPreprocessing0}/{patientID}/{patientID}_T2.nii.gz")

                #FLAIR
                if any(FlairSequence in sequenceLC for FlairSequence in config.FLAIRlist):
                    dicom2nifti.convert_directory(pathToSequence, f"{pathToPreprocessing0}/{patientID}")
                    unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToPreprocessing0}/{patientID}")
                    os.rename(unRenamedFile, f"{pathToPreprocessing0}/{patientID}/{patientID}_FLAIR.nii.gz")

        # move back out
        os.chdir(folderPath)

print("done")

  0%|          | 0/375 [00:00<?, ?it/s]Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series

Anonymized - 02148010


Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
R

Anonymized - 02113718


Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
R

...

 22%|██▏       | 84/375 [02:29<10:44,  2.22s/it]

patientID not 8 numbers long
patientID not 8 numbers long
patientID not 8 numbers long
patientID not 8 numbers long
Anonymized - 01771273


Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
R

patientID not 8 numbers long
patientID not 8 numbers long


TypeError: rename: src should be string, bytes or os.PathLike, not NoneType

### Step 1: Copies patients with the correct amount of sequences (4) into a new directory

In [5]:
#To-do:
# - loop through all the patients from Preprocessing#0
# - only include patients with exactly 4 files and only with FLAIR, T1, T1CE, T2
# - copy files to new directory (i.e. Preprocessing#1)

# Creates a new folder within the directory only for the patients with the correct amount of sequences
# now = datetime.now()
# timeFormatted = now.strftime("%Y-%m-%d %H:%M:%S")
# pathToPreprocessing1 = f"{config.path}/Rgb_Brain_Mets_Preprocessing#1_{timeFormatted}"
# os.mkdir(pathToPreprocessing1)
pathToPreprocessing1 = funcs.createNewPreprocessingStepFolder(step=1)

patientFoldersPreprocessing0 = os.listdir(pathToPreprocessing0)

for patientID in tqdm(patientFoldersPreprocessing0):

    if patientID == ".DS_Store": # ignores the .DS_Store directory
        continue
    
    # list of different files within the patient folder
    patientFiles = os.listdir(f"{pathToPreprocessing0}/{patientID}")

    shouldCopyPatientFiles = False
    
    # check amount of files and correct sequences
    if len(patientFiles) == 4:
        shouldCopyPatientFiles = True
    
    for file in patientFiles:
        sequence = file.split("_")[1] # for example: sequence = FLAIR.nii.gz
        sequenceName = sequence.split(".")[0] # for example: sequenceName = FLAIR
        
        if sequenceName not in config.desiredSequences:
            shouldCopyPatientFiles = False
        

    if shouldCopyPatientFiles:
        # create a new patient directory in pathToPreprocessing1
        pathToPreprocessing1Patient = f"{pathToPreprocessing1}/{patientID}"
        os.mkdir(pathToPreprocessing1Patient)

        # copies files to new directory
        for file in patientFiles:
            shutil.copy2(f"{pathToPreprocessing0}/{patientID}/{file}", pathToPreprocessing1Patient)

#### Export list of patientIDs as a .csv files

In [14]:
patientIDs = os.listdir(pathToPreprocessing1)
patientIDsDataFrame = pd.DataFrame({"patientIDs": []})
patientIDsDataFrame["patientIDs"] = patientIDs
patientIDsDataFrame.to_csv("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/MRI_patientIDs.csv")

### Step 2: Reorientation using FSL
Requires FSL, for further information see https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation

In [6]:
# To-Do:
# - Go trough each patient's files of Preprocessing#1
# - use fslreorient2std

# Creates a new folder within the directory only for the now reoriented MRI sequences
pathToPreprocessing2 = funcs.createNewPreprocessingStepFolder(step=2)

patientFoldersPreprocessing1 = os.listdir(pathToPreprocessing1)

for patientID in tqdm(patientFoldersPreprocessing1):

    if patientID == ".DS_Store": # ignores the .DS_Store directory
        continue
    
    # list of MRI sequences
    patientFiles = os.listdir(f"{pathToPreprocessing1}/{patientID}")
    
    # creates a new folder for each patient in the pathToPreprocessing2
    pathToPreprocessing2Patient = f"{pathToPreprocessing2}/{patientID}"
    os.mkdir(pathToPreprocessing2Patient)

    for file in patientFiles:

        reorientedFileName = file.split(".")[0] + "_reoriented.nii.gz"

        os.system(f"fslreorient2std \"{pathToPreprocessing1}/{patientID}/{file}\" \"{pathToPreprocessing2Patient}/{reorientedFileName}\"")

100%|██████████| 27/27 [00:37<00:00,  1.38s/it]


### Step 3: Brain extraction
https://github.com/MIC-DKFZ/HD-BET

Isensee F, Schell M, Tursunova I, Brugnara G, Bonekamp D, Neuberger U, Wick A, Schlemmer HP, Heiland S, Wick W, Bendszus M, Maier-Hein KH, Kickingereder P. Automated brain extraction of multi-sequence MRI using artificial neural networks. Hum Brain Mapp. 2019; 1–13. https://doi.org/10.1002/hbm.24750

#### Using HD-BET

In [13]:
# To-Do:
# - Go trough each patient's files of Preprocessing#2
# - use hd-bet to extract the brain of the patients

# Creates a new folder within the directory for the files after the brain extraction
pathToPreprocessing3 = funcs.createNewPreprocessingStepFolder(step=3)

patientFoldersPreprocessing2 = os.listdir(pathToPreprocessing2)

for patientID in tqdm(patientFoldersPreprocessing2):
    
    if patientID == ".DS_Store": # ignores the .DS_Store directory
        continue

    # creates a new folder for each patient in the pathToPreprocessing3
    pathToPreprocessing3Patient = f"{pathToPreprocessing3}/{patientID}"
    os.mkdir(pathToPreprocessing3Patient)

    # use hd-bet -i INPUT_FOLDER -o OUTPUT_FOLDER command in the terminal
    #print(f"{pathToPreprocessing2}/{patientID}")
    #print(pathToPreprocessing3Patient)

    os.system(f"hd-bet -i {pathToPreprocessing2}/{patientID} -o {pathToPreprocessing3Patient}")


100%|██████████| 28/28 [00:00<00:00, 10984.99it/s]

/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#2_2023-12-30_18-04-44/01199093
/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#3_2023-12-30_18-14-21/01199093
/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#2_2023-12-30_18-04-44/01220269
/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#3_2023-12-30_18-14-21/01220269
/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#2_2023-12-30_18-04-44/01150136
/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#3_2023-12-30_18-14-21/01150136
/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#2_2023-12-30_18-04-44/01134825
/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Rgb_Brain_Mets/N30/Rgb_Brain_Mets_Preprocessing#3_2023-12-30_18-14-21/0

#### Using SynthStrip
https://surfer.nmr.mgh.harvard.edu/docs/synthstrip/
```
@article{hoopes2022,
    title = {SynthStrip: skull-stripping for any brain image},
    journal = {NeuroImage},
    volume = {260},
    pages = {119474},
    year = {2022},
    issn = {1053-8119},
    doi = {https://doi.org/10.1016/j.neuroimage.2022.119474},
    url = {https://www.sciencedirect.com/science/article/pii/S1053811922005900},
    author = {Andrew Hoopes and Jocelyn S. Mora and Adrian V. Dalca and Bruce Fischl and Malte Hoffmann},
}
```

In [11]:
# To-Do:
# - Go trough each patient's files of Preprocessing#2
# - use synthstrip-docker to extract the brain of the patients

# Creates a new folder within the directory for the files after the brain extraction
pathToPreprocessing3 = funcs.createNewPreprocessingStepFolder(step=3)

patientFoldersPreprocessing2 = os.listdir(pathToPreprocessing2)

for patientID in tqdm(patientFoldersPreprocessing2):
    
    if patientID == ".DS_Store": # ignores the .DS_Store directory
        continue

    # creates a new folder for each patient in the pathToPreprocessing3
    pathToPreprocessing3Patient = f"{pathToPreprocessing3}/{patientID}"
    os.mkdir(pathToPreprocessing3Patient)

    patientFiles = os.listdir(f"{pathToPreprocessing2}/{patientID}")
    
    for file in patientFiles:
        filePatientID = file.split("_")[0]
        fileMRISequences = file.split("_")[1]
        newFileName = f"{filePatientID}_{fileMRISequences}_extracted.nii.gz"

        pathToInputFile = f"{pathToPreprocessing2}/{patientID}/{file}"
        pathToOutputFile = f"{pathToPreprocessing3}/{patientID}/{newFileName}"
        
        os.system(f"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/synthstrip-docker -i {pathToInputFile} -o {pathToOutputFile}")
        

  0%|          | 0/27 [00:00<?, ?it/s]WARNING: The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested


Running SynthStrip from Docker
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#2_2024-01-01_19-32-00/01199093/01199093_T2_reoriented.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#3_2024-01-01_19-45-35/01199093/01199093_T2_extracted.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip

Running SynthStrip from Docker


Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#2_2024-01-01_19-32-00/01199093/01199093_T1_reoriented.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#3_2024-01-01_19-45-35/01199093/01199093_T1_extracted.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip

Running SynthStrip from Docker


Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#2_2024-01-01_19-32-00/01199093/01199093_FLAIR_reoriented.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#3_2024-01-01_19-45-35/01199093/01199093_FLAIR_extracted.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip

Running SynthStrip from Docker


Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#2_2024-01-01_19-32-00/01199093/01199093_T1CE_reoriented.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: /Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification/Rgb_Brain_Mets_Dataset/N30/Rgb_Brain_Mets_Preprocessing#3_2024-01-01_19-45-35/01199093/01199093_T1CE_extracted.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



...

 56%|█████▌    | 15/27 [21:52:45<17:57:11, 5385.97s/it]Traceback (most recent call last):
  File "/freesurfer/env/bin/mri_synthstrip", line 237, in <module>
    sdt = model(input_tensor).cpu().numpy().squeeze()
  File "/freesurfer/env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/freesurfer/env/bin/mri_synthstrip", line 150, in forward
    x = conv(x)
  File "/freesurfer/env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/freesurfer/env/bin/mri_synthstrip", line 188, in forward
    out = self.activation(out)
  File "/freesurfer/env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/freesurfer/env/lib/python3.10/site-packages/torch/nn/modules/activation.py", line 777, in forward
    return F.leaky_relu(input, self.negative_slope, self.in

Running SynthStrip from Docker
Running SynthStrip from Docker
Running SynthStrip from Docker


Running SynthStrip from Docker


 59%|█████▉    | 16/27 [21:52:47<11:30:17, 3765.24s/it]

Running SynthStrip from Docker


### Step 4: Cropping

### Step 5: N4-Bias-Correction

### Step 6: Coregistration

### Step 7: Normalization

### Step 8: Segmentation